<a href="https://colab.research.google.com/github/mymph/modelos-ml-rp/blob/main/submition_rp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Dupla: Letícia Lívia e Rodrigo Leandro

# Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_selection import RFE

# Carregamento dos Dados

In [ ]:
train_val_df = pd.read_csv('/content/projeto_va1_base_de_dados_train_val.csv')
test_df = pd.read_csv('/content/projeto_va1_base_de_dados_teste_sem_target.csv')

#verificar os primeiros registros
train_val_df.head()

,UF,SEXO,QT_DIARIAS,VAL_UTI,IDADE,INSTRU,RACA_COR,BODY_SYSTEM,H_COST
0,31,3,1,0.0,4,0,99,8,0
1,26,3,15,0.0,71,0,3,2,0
2,35,3,30,0.0,61,0,1,5,0
3,23,3,1,0.0,50,0,3,10,0
4,43,1,24,0.0,38,0,1,5,0


#Análise Exploratória de Dados

In [ ]:
#verificar os tipos de dados em cada coluna no caso de alguma precisar de codificação
print(train_val_df.dtypes)

UF               int64
SEXO             int64
QT_DIARIAS       int64
VAL_UTI        float64
IDADE            int64
INSTRU           int64
RACA_COR         int64
BODY_SYSTEM      int64
H_COST           int64
dtype: object


In [ ]:
#verificar se é necessário tratar valores ausentes
print(train_val_df.isna().sum())

UF             0
SEXO           0
QT_DIARIAS     0
VAL_UTI        0
IDADE          0
INSTRU         0
RACA_COR       0
BODY_SYSTEM    0
H_COST         0
dtype: int64


#Pré-processamento de Dados

In [ ]:
# Tratamento de Missing Values

train_val_df.fillna(train_val_df.mode(), inplace=True)
test_df.fillna(test_df.median(), inplace=True)

In [ ]:
# Codificação de Variáveis Categóricas

label_encoders = {}
for column in ['SEXO', 'INSTRU', 'RACA_COR', 'BODY_SYSTEM']:
    le = LabelEncoder()
    train_val_df[column] = le.fit_transform(train_val_df[column])
    test_df[column] = le.transform(test_df[column].map(lambda s: '<unknown>' if s not in le.classes_ else s))
    le.classes_ = np.append(le.classes_, '<unknown>')
    label_encoders[column] = le

# Separação das Features e Target

In [ ]:
X = train_val_df.drop(columns=['H_COST'])
y = train_val_df['H_COST']

In [ ]:
# primeiras linhas das variáveis independentes `x` (features)
print(X.head())

print("\n")

# primeiras linhas da variável dependente `y` (target)
print(y.head())

   UF  SEXO  QT_DIARIAS  VAL_UTI  IDADE  INSTRU  RACA_COR  BODY_SYSTEM
0  31     3           1      0.0      4       0        99            8
1  26     3          15      0.0     71       0         3            2
2  35     3          30      0.0     61       0         1            5
3  23     3           1      0.0     50       0         3           10
4  43     1          24      0.0     38       0         1            5


0    0
1    0
2    0
3    0
4    0
Name: H_COST, dtype: int64


# Divisão em conjunto de Treinamento e Validação

In [ ]:
# Dividir os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# X_train (features de treinamento)
print(X_train.head())

print("\n")

# X_val (features de validação)
print(X_val.head())

print("\n")

# y_train (target de treinamento)
print(y_train.head())

print("\n")

# y_val (target de treinamento)
print(y_val.head())

         UF  SEXO  QT_DIARIAS  VAL_UTI  IDADE  INSTRU  RACA_COR  BODY_SYSTEM
2179118  21     1           2      0.0     63       0        99            7
3010952  26     1           4      0.0     67       0         3            5
1185789  29     3           1      0.0     74       0         3            6
1632198  31     3           7      0.0     75       0         3           10
3367111  42     1           4      0.0     66       0         1            2


         UF  SEXO  QT_DIARIAS  VAL_UTI  IDADE  INSTRU  RACA_COR  BODY_SYSTEM
367692   31     1           1      0.0     34       0         1            7
105116   35     3           2      0.0     57       0         1            2
2311018  35     1          31      0.0     31       0         1            5
3282034  35     3           2      0.0     25       0         1           11
2831660  33     1           9      0.0     66       0         1            7


2179118    0
3010952    0
1185789    0
1632198    0
3367111    0
Name: H

# Normalização

In [ ]:
#criação do scaler p todas as colunas
all_features = X_train.columns
scaler = StandardScaler()

#aplicação da transformação
X_train[all_features] = scaler.fit_transform(X_train[all_features])
X_val[all_features] = scaler.transform(X_val[all_features])
test_df[all_features] = scaler.transform(test_df[all_features])

# Feature Selection com RFE

In [ ]:
#modelo base para RFE para KNN (os outros modelos usados para teste não)
base_model = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

#RFE para selecionar as melhores 10 features
selector = RFE(base_model, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)

X_train_selected = selector.transform(X_train)
X_val_selected = selector.transform(X_val)
X_test_selected = selector.transform(test_df)

#verificar quais features foram selecionadas
selected_features = np.array(all_features)[selector.support_]
print(f"Selected features: {selected_features}")

Selected features: ['UF' 'SEXO' 'QT_DIARIAS' 'VAL_UTI' 'IDADE' 'INSTRU' 'RACA_COR'
 'BODY_SYSTEM']


# Construção do Modelo e ajuste dos Hiperparâmetros

In [ ]:
#construção e treino do modelo de Regressão Logística
log_reg_model = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=1000)
log_reg_model.fit(X_train, y_train)

#previsões e métricas
y_val_pred_lr = log_reg_model.predict(X_val)
acc_lr = accuracy_score(y_val, y_val_pred_lr)
f1_lr = f1_score(y_val, y_val_pred_lr)
print(f'Regressão Logística - Acurácia: {acc_lr:.4f}')
print(classification_report(y_val, y_val_pred_lr))

Regressão Logística - Acurácia: 0.9560
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    702293
           1       0.68      0.24      0.36     37235

    accuracy                           0.96    739528
   macro avg       0.82      0.62      0.67    739528
weighted avg       0.95      0.96      0.95    739528



In [ ]:
#construção e treino do modelo Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

#previsões e métricas
y_val_pred_nb = nb_model.predict(X_val)
acc_nb = accuracy_score(y_val, y_val_pred_nb)
f1_nb = f1_score(y_val, y_val_pred_nb)
print(f'Naive Bayes - Acurácia: {acc_nb:.4f}')
print(classification_report(y_val, y_val_pred_nb))

Naive Bayes - Acurácia: 0.0505
              precision    recall  f1-score   support

           0       1.00      0.00      0.00    702293
           1       0.05      1.00      0.10     37235

    accuracy                           0.05    739528
   macro avg       0.53      0.50      0.05    739528
weighted avg       0.95      0.05      0.01    739528



In [ ]:
#construção e treino do modelo KNN com RFE
knn_model = KNeighborsClassifier(n_neighbors=20, metric='euclidean')
knn_model.fit(X_train_selected, y_train)

#previsões e métricas
y_val_pred_knn = knn_model.predict(X_val_selected)
acc_knn = accuracy_score(y_val, y_val_pred_knn)
f1_knn = f1_score(y_val, y_val_pred_knn)
print(f'KNN com RFE - Accurácia: {acc_knn:.4f}')
print(classification_report(y_val, y_val_pred_knn))

KNN com RFE - Accurácia: 0.9610
              precision    recall  f1-score   support

           0       0.97      0.99      0.98    702293
           1       0.72      0.36      0.48     37235

    accuracy                           0.96    739528
   macro avg       0.85      0.68      0.73    739528
weighted avg       0.95      0.96      0.95    739528



In [ ]:
#selecionar o melhor modelo baseado no f1-score
best_model = None
best_score = 0

if f1_lr > best_score:
    best_score = f1_lr
    best_model = log_reg_model

if f1_nb > best_score:
    best_score = f1_nb
    best_model = nb_model

if f1_knn > best_score:
    best_score = f1_knn
    best_model = knn_model

print(f'\nMelhor modelo: {best_model}')


Melhor modelo: KNeighborsClassifier(metric='euclidean', n_neighbors=20)


# Avaliação no conjunto de Validação

In [ ]:
#avaliação no conjunto de Validação

y_probs = best_model.predict_proba(X_val_selected)[:, 1]

threshold = 0.4
y_pred = (y_probs >= threshold).astype(int)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98    702293
           1       0.60      0.50      0.54     37235

    accuracy                           0.96    739528
   macro avg       0.78      0.74      0.76    739528
weighted avg       0.95      0.96      0.96    739528



# Previsão no conjunto de Teste

In [ ]:
#previsão com o melhor modelo
y_test_pred = best_model.predict(X_test_selected)

#contagem das previsões no conjunto de teste
prediction_counts = pd.Series(y_test_pred).value_counts()
print("Contagem de cada previsão no conjunto de teste:")
print(prediction_counts)

Contagem de cada previsão no conjunto de teste:
0    1544356
1      40347
Name: count, dtype: int64


# Criação do arquivo de Submissão

In [ ]:
#salvar resultado
output = pd.DataFrame({'H_COST_PRED': y_test_pred})
output.to_csv('/content/submission.csv', index=False)

print("Previsões salvas em 'submission.csv'")

Previsões salvas em 'submission.csv'
